# Toy Radix Benders Multicut

In [1]:
LP_RELAXATION = True
PREVENT_ZERO = True
REG_WEIGHT = 0. #1e-4    # Regularization weight
MAX_NONZERO  = None #48*2
MULTICUT = True
NONDOM   = True
NOGOOD   = False

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
mdl_ref = ijomc

ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [5]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

# N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [6]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# 0) DEBUG: test using single problem

df_X = df_conds
df_Y = df_meas[['substrate','growth_rate_1_h']].rename(columns={'substrate':'cond','growth_rate_1_h':'output'})
df_Y.loc[:,'output_id'] = 'BIOMASS_Ecoli_core_w_GAM'

from dynamicme.estimate import RadixEstimator
base_model = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
est = RadixEstimator()
est.fit(base_model, df_X, df_Y, optimize=False, reg_weight=1e-4)
est.optimize()

# 0) Load changed keffs to reduce binary vars

In [7]:
import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json') as f:
    kfit_changed = json.load(f)

In [8]:
from dynamicme.decomposition import BendersSubmodel, BendersMaster
from dynamicme.generate import copy_model

In [9]:
changed_keffs = [kv[0] for kv in kfit_changed]

In [10]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    sn.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:
        if rxn_ref.id in changed_keffs:
            crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
            var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
            rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
            cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
            a0 = rxn_p.metabolites[crowding_p]
            if var_cons_dict.has_key(rxn_ref.id):
                var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
            else:
                var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = BendersSubmodel(gap, cond)
    sub_dict[cond] = sub
    
for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = REG_WEIGHT

master = BendersMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [11]:
master.add_submodels(sub_dict)

In [12]:
# mdl1 = sub_dict['acetate'].cobra_model
# mdl2 = sub_dict['glucose'].cobra_model
# for rxn1 in mdl1.reactions:
#     if mdl2.reactions.has_id(rxn1.id):
#         rxn2 = mdl2.reactions.get_by_id(rxn1.id)
#         print(rxn1,rxn2, rxn1==rxn2)

In [13]:
from dynamicme.callback_gurobi import cb_benders_multi, cb_benders

# Solve two-phase

### Know a feasible point, get core point

### Initial core point: y_jkl=1 for digit(k)=1, power(l)=0 forall j
nkeffs = len(var_cons_dict)

var_ind = {v:j for j,v in enumerate(master._ys)}
y0 = np.ones(len(master._ys))
for r in var_cons_dict.keys():
    for k in range(len(digits)):
        for l in range(len(powers)):
            var = master.model.getVarByName('binary_%s%s%s'%(r,k,l))
            ind = var_ind[var]
            if digits[k]==1 and powers[l]==0:                
                y0[ind] = 1
            else:
                y0[ind] = 0
            ### Also use as initial feasible solution (and possibly incumbent)
            var.Start = y0[ind]
sum(y0)

y2 = np.array([0.5*(y.LB+y.UB) for y in master._ys])
y02 = 0.5*(y0+y2)
#y0 = 0.5*(y0 + np.ones(len(master._ys)))
master.y0 = y02

In [14]:
### Make it easer: z and tk are absolute error or sum w*y so >=0
master._z.LB = 0.
for sub_ind in sub_dict.keys():
    tk = master.model.getVarByName('tk_%s'%sub_ind)
    tk.LB = 0.
master.model.update()

In [15]:
master.model.Params.NumericFocus = 3
master.model.Params.OutputFlag = 1
#master.model.Params.ScaleFlag = 0
master.model.Params.Presolve = 0   # Don't let presolve remove rows
master._verbosity = 1
master.print_iter = 20

# master.solve_relaxed()
# master.precision_sub = 'dq'
master.precision_sub = 'gurobi'
master.max_iter = 1000
yopt = master.optimize(two_phase=True, cut_strategy='mw')

Changed value of parameter NumericFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
        Iter          UB          LB     Best UB     Best LB         gap   relgap(%)     time(s)
           0       1e+15      -1e+15       1e+15      -1e+15       2e+15       200.0    0.000117
          20    0.552548         0.0    0.377749         0.0    0.552548  99.9999999    0.351103
          40    0.552548         0.0    0.377749         0.0    0.552548  99.9999999    0.638665
          60    0.552548         0.0    0.377749         0.0    0.552548  99.9999999    0.936886
          80    0.552548         0.0    0.377749         0.0    0.552548  99.9999999    1.224991
         100    0.552548         0.0    0.377749         0.0    0.552548  99.9999999    1.519382
         120    0.552548         0.0    0.377749         0.0    0.552548  99.999999

In [16]:
sub.model.model.update()
sub.model.optimize(precision='quad')

In [17]:
sub.model.ObjVal

0.17447980200228194

In [18]:
print('optcuts:', len(master.optcuts))
print('feascuts:',len(master.feascuts))

('optcuts:', 8)
('feascuts:', 0)


In [19]:
sum(master.int_sols)/len(master.int_sols)

array([0.33333333, 0.66666667, 1.        , 0.33333333, 0.66666667,
       1.        , 0.33333333, 0.66666667, 1.        , 0.66666667,
       0.33333333, 0.66666667, 0.33333333, 0.66666667, 1.        ,
       0.33333333, 0.66666667, 1.        , 0.33333333, 0.66666667,
       0.66666667, 0.33333333, 0.66666667, 1.        , 0.33333333,
       0.66666667, 1.        , 0.66666667, 0.33333333, 0.66666667,
       0.66666667, 0.33333333, 0.66666667, 1.        , 0.33333333,
       0.33333333, 0.66666667, 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.33333333, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.33333333, 0.66666667, 0.66666667, 0.33333333,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.33333333,
       0.33333333, 1.        , 0.66666667, 0.66666667, 0.33333333,
       0.33333333, 1.        , 0.33333333, 0.33333333, 0.        ,
       0.66666667])

In [20]:
len(master.int_sols)

3

In [21]:
c = sub.model.model.getConstrByName('fixobjval')
print(c.RHS)
sub.model.model.getRow(c)

1e+100


<gurobi.LinExpr: 0.001 wa[72] + wa[86] + 0.256 wa[169] + 1000.0 wa[170] + 1000.0 wa[171] + 1000.0 wa[172] + 1000.0 wa[173] + 1000.0 wa[174] + 1000.0 wa[175] + 1000.0 wa[177] + 1000.0 wa[178] + 1000.0 wa[179] + 1000.0 wa[180] + 1000.0 wa[181] + 1000.0 wa[182] + 1000.0 wa[183] + 1000.0 wa[185] + 1000.0 wa[186] + 1000.0 wa[187] + 1000.0 wa[188] + 1000.0 wa[189] + 1000.0 wa[190] + 1000.0 wa[191] + 1000.0 wa[193] + 1000.0 wa[194] + 1000.0 wa[195] + 1000.0 wa[197] + 1000.0 wa[198] + 1000.0 wa[199] + 1000.0 wa[201] + 1000.0 wa[202] + 1000.0 wa[203] + 1000.0 wa[205] + 1000.0 wa[206] + 1000.0 wa[207] + 1000.0 wa[209] + 1000.0 wa[210] + 1000.0 wa[211] + 1000.0 wa[213] + 1000.0 wa[214] + 1000.0 wa[215] + 1000.0 wa[217] + 1000.0 wa[218] + 1000.0 wa[219] + 1000.0 wa[221] + 1000.0 wa[222] + 1000.0 wa[223] + 1000.0 wa[225] + 1000.0 wa[226] + 1000.0 wa[227] + 1000.0 wa[229] + 1000.0 wa[230] + 1000.0 wa[231] + 1000.0 wa[233] + 1000.0 wa[234] + 1000.0 wa[235] + 1000.0 wa[237] + 1000.0 wa[238] + 1000.0 w

In [22]:
sub.model.model.getObjective()

<gurobi.LinExpr: 0.001 wa[72] + wa[86] + 0.256 wa[169] + 906.25 wa[170] + 906.25 wa[171] + 93.75 wa[172] + 93.75 wa[173] + 906.25 wa[174] + 906.25 wa[175] + 93.75 wa[177] + 531.25 wa[178] + 531.25 wa[179] + 468.75 wa[180] + 468.75 wa[181] + 531.25 wa[182] + 531.25 wa[183] + 468.75 wa[185] + 1000.0 wa[188] + 1000.0 wa[189] + 1000.0 wa[193] + 875.0 wa[194] + 875.0 wa[195] + 125.0 wa[197] + 875.0 wa[198] + 875.0 wa[199] + 125.0 wa[201] + 531.25 wa[202] + 531.25 wa[203] + 468.75 wa[205] + 531.25 wa[206] + 531.25 wa[207] + 468.75 wa[209] + 31.25 wa[210] + 31.25 wa[211] + 968.75 wa[213] + 31.25 wa[214] + 31.25 wa[215] + 968.75 wa[217] + 906.25 wa[218] + 906.25 wa[219] + 93.75 wa[221] + 906.25 wa[222] + 906.25 wa[223] + 93.75 wa[225] + 531.25 wa[226] + 531.25 wa[227] + 468.75 wa[229] + 531.25 wa[230] + 531.25 wa[231] + 468.75 wa[233] + 1000.0 wa[237] + 1000.0 wa[241] + 531.25 wa[242] + 531.25 wa[243] + 468.75 wa[244] + 468.75 wa[245] + 531.25 wa[246] + 531.25 wa[247] + 468.75 wa[249] + 906.25

In [23]:
master.y0

array([0.046875, 0.234375, 1.      , 0.0625  , 0.234375, 0.984375,
       0.046875, 0.234375, 1.      , 0.234375, 0.046875, 0.8125  ,
       0.046875, 0.234375, 1.      , 0.046875, 0.234375, 1.      ,
       0.046875, 0.234375, 0.8125  , 0.046875, 0.234375, 1.      ,
       0.046875, 0.25    , 0.984375, 0.234375, 0.0625  , 0.796875,
       0.234375, 0.046875, 0.8125  , 0.984375, 0.0625  , 0.046875,
       0.25    , 0.046875, 0.796875, 0.234375, 0.796875, 0.0625  ,
       0.234375, 0.234375, 0.8125  , 0.234375, 0.046875, 0.8125  ,
       0.234375, 0.046875, 0.8125  , 0.234375, 0.25    , 0.796875,
       0.046875, 0.046875, 1.      , 0.25    , 0.796875, 0.046875,
       0.0625  , 0.984375, 0.046875, 0.046875, 0.015625, 0.9375  ])

from dynamicme.callback_gurobi import constraint_satisfied
x_dict = {v:v.X for v in master.model.getVars()}
for c in list(master.optcuts) + list(master.feascuts):
    print(constraint_satisfied(c, x_dict, master.model.Params.FeasibilityTol))
    if not constraint_satisfied(c, x_dict, master.model.Params.FeasibilityTol):
        print(c)

In [24]:
# x_dict

In [25]:
# yopt = master.solve_loop()

In [26]:
#master.model.Params.Presolve = 0
#master.model.Params.ScaleFlag = 0
#master.model.Params.Heuristics = 0   # cbLazy might be bugged by heuristics

master.model.Params.OutputFlag = 1
master._verbosity = 1
master.model.optimize(cb_benders_multi)

## For faster debugging, only keep binary for known keff changes

### Initial core point: y_jkl=1 for digit(k)=1, power(l)=0 forall j
nkeffs = len(var_cons_dict)

var_ind = {v:j for j,v in enumerate(master._ys)}
y0 = np.ones(len(master._ys))
for r in var_cons_dict.keys():
    for k in range(len(digits)):
        for l in range(len(powers)):
            var = master.model.getVarByName('binary_%s%s%s'%(r,k,l))
            ind = var_ind[var]
            if digits[k]==1 and powers[l]==0:                
                y0[ind] = 1
            else:
                y0[ind] = 0
            ### Also use as initial feasible solution (and possibly incumbent)
            var.Start = y0[ind]
sum(y0)

def opt_nondom(sub, y0):
    # Add or update the constraint for same obj
    cons = sub.model.model.getConstrByName('fixobjval')
    if cons is None:
        expr = sub.model.model.getObjective() == sub.model.model.ObjVal
        cons = sub.model.model.addConstr(expr, name='fixobjval')
    else:
        cons.RHS = sub.model.model.ObjVal
        obj = sub.model.model.getObjective()
        for j in range(obj.size()):
            v = obj.getVar(j)
            sub.model.model.chgCoeff(cons, v, obj.getCoeff(j))
    # Change the objective function
    sub.update_obj(y0)
    sub.model.optimize()
    # Relax the constraint for next iter
    cons.Sense = GRB.LESS_EQUAL
    cons.RHS   = GRB.INFINITY

In [27]:
sol_master = {x.VarName:x.X for x in master.model.getVars()}
#yopt = [sol_master[y.VarName] for y in master._ys]
print('Number of non-zero binaries: %g' % sum(yopt))
# Fitted parameters

Number of non-zero binaries: 22


sol_master = est.solution.x_dict
var_cons_dict = est.var_cons_dict
powers = est.powers
digits = est.digits
radix  = est.radix

In [28]:
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))            
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
# print('kfit_changed:',kfit_changed)

#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:    
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Changed keffs: 19/22
Cond=glucose. mu_meas=0.74. mu_sim=0.873922 (unfit=0.873922, error=18.1%). Error=18.1% (-1.78e-10% change)
	EX_glc__D_e uptake=-10
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.389313 (unfit=0.389313, error=52.1%). Error=52.1% (4.64e-11% change)
	EX_glc__D_e uptake=0
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=-20


import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json','w') as f:
    json.dump(kfit_changed,f)